In [1]:
import cv2
import glob
import pandas as pd
import numpy as np
from imputation import Imputation
from geotiff import GeoTiff as gt
from osgeo import gdal, ogr

In [2]:
# import helpers

# corr_folder = [file for file in glob.glob("./bad/*")]
# corr = []
# for file_name in corr_folder:
#     image = cv2.imread(file_name)
#     img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     # with io.open(file_name, 'rb') as image_file:
#     #     content = image_file.read()
#     corr.append(img)


# orig_folder = [file for file in glob.glob("./good/*")]
# orig = []
# for file_name in corr_folder:
#     image = cv2.imread(file_name)
#     img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     # with io.open(file_name, 'rb') as image_file:
#     #     content = image_file.read()
#     orig.append(img)

# corr = np.array(corr)
# orig = np.array(orig)
# cmt_arr = []
# cp_arr = []

def readTIF(path):
    dataset = gdal.Open(path)
    print(dataset.RasterCount)
    geotrans=dataset.GetGeoTransform()
    proj=dataset.GetProjection()
    image=dataset.ReadAsArray()
    return (geotrans,proj,image)



In [3]:
# for i in range(len(corr)):
#     imp = Imputation(np.array(corr[i]))
#     imp.generate_mask()
#     imp.impute(speed="fast",skip_vis=1)
#     cmt_arr.append(helpers.MSE(imp.cmtf,orig[i]))
#     cp_arr.append(helpers.MSE(imp.cpals,orig[i]))


In [4]:
# #RESULTS
# print(cmt_arr)
# print(cp_arr)
# print('')

In [5]:
# df = pd.read_csv('../MISC/test.tsv', sep='\t', header=None)
# imp = Imputation(df)

In [6]:
tifread = readTIF('/home/shiridi/research_work/Lunar_image_reconstruction/kaguya_dataset_striped/Area1_MI_MAP_03_N22E196N21E197SC.tif')
img = tifread[-1]

imp = Imputation(img)
# print(tifread.read())

9


In [7]:
# [IMP] need to use this function to set
# the brightness for interactive parts/visualization
imp.set_brightness(0.0007) # 1 1000 100 0.1
# imp.show_img(imp.img)

In [8]:
# MASK: 1 for valid pixel, 0 for missing pixels

# imp.generate_mask(threshold): this threshold value translates to the 
# minimum difference between neighboring pixels which would cause one of them
# to be identified as corrupt pixels. Leaving it blank will generate & print
# an auto threshold which will be the average of all the values.
# [IMP: This thresholding value depends entirely on the dataframe]


# imp.generate_mask() # 1. to find a mask for an image with missing pixels

# imp.generate_mask(approximate_mask=True) # 2. to find an approximate mask for an image with missing pixels

imp.synthetic_mask(1.0) # 3. to randomly delete pixels

# 'r' to reset, 'enter' to continue after drawing mask, 'o/p' to decrease/increase size of cursor
# imp.draw_mask() # 4. Draw a mask. LClick: freehand drawing, RClick: draw a rect

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [9]:
# hil = 0->no HiL, 1->run HiL, 2->run ONLY HiL
imp.impute(speed="slow", hil=1) # hiL is for CMTF4SI

Imputation val =  11082.856844610284
HaLRTC:
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 

KeyboardInterrupt: 

In [13]:
test = imp.get_best_np() # np array of the best imputed image

In [25]:
# NOTE: Please consider RSE values only when the original image is NOT corupt.
# Otherwise those values are meaningless.
imp.visualize() 

Exception: Imputation not done yet!

In [32]:
def CreateGeoTiff(outRaster, data, geo_transform, projection):
    driver = gdal.GetDriverByName('GTiff')
    no_bands,rows,cols = data.shape
    DataSet = driver.Create(outRaster, cols, rows, no_bands, gdal.GDT_Float32)
    DataSet.SetGeoTransform(geo_transform)
    DataSet.SetProjection(projection)
    for i, image in enumerate(data, 1):
        # print(i,image)
        DataSet.GetRasterBand(i).WriteArray(image)
    DataSet.FlushCache()
    DataSet = None
import os
os.system("mkdir kaguga_silRTC_20")
CreateGeoTiff("kaguga_silRTC_20/Area1_MI_MAP_03_N22E196N21E197SC.tif",imp.sil,tifread[0],tifread[1])